In [1]:
%load_ext autoreload

%autoreload 2

from RNN2DFA.LSTM import LSTMNetwork
# from GRU import GRUNetwork
from RNN2DFA.RNNClassifier import RNNClassifier
from RNN2DFA.Training_Functions import mixed_curriculum_train
import Tomita_Grammars 
from RNN2DFA.Training_Functions import make_test_set,make_train_set_for_target
from RNNexplainer import Explainer
import pandas as pd
import LTL2DFA as ltlf2dfa





In [2]:
from specific_examples import Alternating_Bit_Protocol
abp=Alternating_Bit_Protocol()
generator_dfa=abp.dfa
target_formula=abp.target_formula
alphabet=generator_dfa.alphabet



In [3]:
from specific_examples import Email
email=Email()
generator_dfa=email # to make everything work
target_formula=email.target_formula
alphabet=email.alphabet


In [4]:
# Reber grammar
from specific_examples import Reber_Grammar
rg=Reber_Grammar()
alphabet=rg.alphabet
generator_dfa=rg
sample_train_set=[]
for i in range(100):
    seq, _, _ = rg.get_one_example(maxLength=10)
    sample_train_set.append(seq)
    # print(rg.classify_word(rg.sequenceToWord(seq)))

In [5]:
# balanced parentheses
from specific_examples import Balanced_Parentheses
bp=Balanced_Parentheses()
bp.get_balanced_parantheses_train_set(10,1,10)
alphabet=bp.alphabet

made train set of size: 1169 , of which positive examples: 424


In [27]:




# use a dfa to generate training set
target_formula="F(l U (~l))"
alphabet=bp.alphabet
import LTL2DFA as ltlf2dfa
generator_dfa=ltlf2dfa.translate_ltl2dfa(alphabet=[character for character in alphabet],formula=target_formula)
print(generator_dfa)


DFA:->
 - alphabet: ['l', 'r', 'a', 'b', 'c', 'd']
 - Q: [1, 2, 3, 4]
 - q0: 1
 - F: [4]
 - delta: {1: {'000000': 2, '000001': 2, '000010': 2, '000011': 2, '000100': 2, '000101': 2, '000110': 2, '000111': 2, '001000': 2, '001001': 2, '001010': 2, '001011': 2, '001100': 2, '001101': 2, '001110': 2, '001111': 2, '010000': 2, '010001': 2, '010010': 2, '010011': 2, '010100': 2, '010101': 2, '010110': 2, '010111': 2, '011000': 2, '011001': 2, '011010': 2, '011011': 2, '011100': 2, '011101': 2, '011110': 2, '011111': 2, '100000': 3, '100001': 3, '100010': 3, '100011': 3, '100100': 3, '100101': 3, '100110': 3, '100111': 3, '101000': 3, '101001': 3, '101010': 3, '101011': 3, '101100': 3, '101101': 3, '101110': 3, '101111': 3, '110000': 4, '110001': 4, '110010': 4, '110011': 4, '110100': 4, '110101': 4, '110110': 4, '110111': 4, '111000': 4, '111001': 4, '111010': 4, '111011': 4, '111100': 4, '111101': 4, '111110': 4, '111111': 4}, 2: {'000000': 2, '000001': 2, '000010': 2, '000011': 2, '000100

In [30]:
generator_dfa.classify_word("lrl")

True

In [5]:
fout=open("output/log.txt", "a")
fout.write(".........................................................................\n")
fout.write("Target: "+ target_formula)
fout.write("\n")
fout.close()

In [6]:
# make training set
train_set = make_train_set_for_target(generator_dfa.classify_word,alphabet,max_train_samples_per_length=1000,search_size_per_length=3000)
# print(train_set)

made train set of size: 10860 , of which positive examples: 5427


In [7]:
print(train_set)

bbcccc': False, 'acbccbbbcccacbbacbba': False, 'bbcbcbaaacbcbcaccaca': False, 'cbcacbacbaccccbaacba': False, 'cbbccbcbccaacaaacaca': False, 'bbbbcbacccaaacaacacc': False, 'ccaacbcbcbacbcacacbb': False, 'cbccbcccbcccacacbacb': False, 'ccbcccbccbcacbacccbb': False, 'accaccbcccbbaaacbaac': False, 'bcaaacbbbaccaaccaaca': False, 'bbcbcbacaacbbcccbacb': False, 'bacacbcaccaaccaaacaa': False, 'accacaaccbcccbacbcba': False, 'cacacbcbbcccccbbbbbc': False, 'aacbbcbcbbacaaaccbbc': False, 'bbbbbccbacccbbcbcccc': False, 'aacbbbbacbcacaacacbb': False, 'cacbcaaccccaaacbbcac': False, 'bbcbacbaccacccaccbcb': False, 'bcacaaaacbcaaacbcbac': False, 'cbcccbcbacaaacaaacbb': False, 'caacbcaacbbbccbccbbc': False, 'bbaaacbbbaacacbbaccc': False, 'cbbacacbbbccbcccbcbc': False, 'accbcaaacbccacccbaca': False, 'cbccbbccaaacbacccbba': False, 'bbcbbcbcaccacbcbbaac': False, 'baacaccacbbbbbaacaca': False, 'bccbcacaccccbcacaaaa': False, 'caaccbaacbcbcbcacaca': False, 'cbaccbbcbbcbccccbccb': False, 'cbbbbcaccaaccbbbbbca':

In [8]:
print(len(train_set))
if(target_formula=="email match"):
    matching_strings=email.generate_matching_strings(1000,20)
    for string in matching_strings:
        train_set[string]=True
print(len(train_set))

10860
10860


In [17]:


# define rnn
rnn = RNNClassifier(alphabet,num_layers=1,hidden_dim=10,RNNClass = LSTMNetwork)


# train the model
mixed_curriculum_train(rnn,train_set,stop_threshold = 0.0005)
rnn.renew()  
dfa_from_rnn=rnn 
# statistics

def percent(num,digits=2):
    tens = pow(10,digits)
    return int(100*num*tens)/tens

print("testing on train set, i.e. test set is train set")
# we're printing stats on the train set for now, but you can define other test sets by using
# make_train_set_for_target

n = len(train_set)
print("test set size:", n)
pos = 0
rnn_target = 0
for w in train_set:
    if generator_dfa.classify_word(w):
        pos+=1

    if dfa_from_rnn.classify_word(w)==generator_dfa.classify_word(w):
        rnn_target+=1
print("rnn score against target on test set:                             ",rnn_target,"("+str(percent(rnn_target/n))+")")

# dfa_from_rnn=generator_dfa

classification loss on last batch was: 0.00046116977508358254
testing on train set, i.e. test set is train set
test set size: 10860
rnn score against target on test set:                              10860 (100.0)


In [19]:


# use a query LTL formula
query_formula="true"
query_dfa=ltlf2dfa.translate_ltl2dfa(alphabet=[character for character in alphabet],formula=query_formula)

"""  
Create initial samples
"""


from RNNexplainer import Traces
traces=Traces(rnn, alphabet)
traces.label_from_network([])
traces.write_in_file()




from PACTeacher.pac_teacher import PACTeacher as Teacher 
explainer=Explainer(alphabet=[character for character in alphabet])
teacher = Teacher(dfa_from_rnn,epsilon=.05, delta=.05, max_trace_length=20, max_formula_depth=10, query_dfa=query_dfa)



import time
start_time=time.time()
from multiprocessing import Process, Queue
explainer, flag= teacher.teach(explainer, traces, timeout = 15)
end_time=time.time()


print("\n\nepsilon=", teacher.epsilon, "delta=", teacher.delta, "max_trace_length=", teacher.max_trace_length)
print("query:", query_formula)
print("final ltl: ", explainer.ltl)

fout=open("output/log.txt", "a")
fout.write("\n\nquery: "+query_formula)
fout.write("\nfinal LTL: "+ explainer.ltl)
if(not flag):
    fout.write(" [incomplete]")
    print("incomplete formula")
fout.write("\n\n")

print("\nTime taken:", end_time-start_time)
fout.close()


fout=open("output/log.txt", "a")
# fout.write("rnn score against target on test set:                             "+str(rnn_target)+"("+str(percent(rnn_target/n))+")")
# fout.write("\n")

performance_ltl_wo_query = performance_ltl_with_target_wo_query = performance_ltl = performance_ltl_with_target = 0


for w in train_set:
    if dfa_from_rnn.classify_word(w)==explainer.dfa.classify_word(w):
        performance_ltl_wo_query+=1
    if explainer.dfa.classify_word(w)==generator_dfa.classify_word(w):
        performance_ltl_with_target_wo_query +=1
    if (dfa_from_rnn.classify_word(w)and query_dfa.classify_word(w)) ==explainer.dfa.classify_word(w):
        performance_ltl+=1
    if explainer.dfa.classify_word(w)== (generator_dfa.classify_word(w) and query_dfa.classify_word(w)):
        performance_ltl_with_target +=1

print("extracted LTL score against rnn on test set:                      ",performance_ltl_wo_query,"("+str(percent(performance_ltl_wo_query/n))+")")

print("extracted LTL score against target on rnn's test set:             ",performance_ltl_with_target_wo_query,"("+str(percent(performance_ltl_with_target_wo_query/n))+")")

print("extracted LTL score against rnn on test set (with query):         ",performance_ltl,"("+str(percent(performance_ltl/n))+")")

print("extracted LTL score against target on rnn's test set (with query):",performance_ltl_with_target,"("+str(percent(performance_ltl_with_target/n))+")")

fout.close()


# report in a pandas file
result = pd.DataFrame(columns=['target', 
                                'query', 
                                'explanation', 
                                'status', 
                                'rnn score', 
                                'explanation score', 
                                'explanation score on ground truth',
                                'extraction time'
                                ])

result = result.append(
    {
        'target':target_formula,
        'query':query_formula,
        'explanation':explainer.ltl,
        'status':flag,
        'rnn score':percent(rnn_target/n),
        'explanation score':percent(performance_ltl/n),
        'explanation score on ground truth':percent(performance_ltl_with_target/n),
        'extraction time': end_time-start_time
    }, ignore_index=True
)
print(result.to_string(index=False))
result.to_csv('output/result.csv', header=False, index=False, mode='a')




positive traces---> 
[]


negative traces---> 
[]



learned LTL formula: true
Learning took:  0.04761385917663574  s
EQ test took  0.0027091503143310547  s
new counterexample:   should be rejected by implementation


positive traces---> 
[]


negative traces---> 
['']



0  iteration complete



learned LTL formula: false
Learning took:  0.06301116943359375  s
EQ test took  0.039186954498291016  s
new counterexample: ab  should be accepted by implementation


positive traces---> 
['ab']


negative traces---> 
['']



1  iteration complete



start formula depth: 1
learned LTL formula: a
Learning took:  0.0792539119720459  s
EQ test took  0.007261991500854492  s
new counterexample: a  should be rejected by implementation


positive traces---> 
['ab']


negative traces---> 
['', 'a']



2  iteration complete



start formula depth: 1
learned LTL formula: (X b)
Learning took:  0.12215614318847656  s
EQ test took  0.01376795768737793  s
new counterexample: bab  should be accepted by imp

In [8]:
# read result
# df=pd.read_csv("output/result.csv",header=None)
# df.columns=['target', 
#             'query', 
#             'explanation', 
#             'status', 
#             'rnn score', 
#             'explanation score', 
#             'explanation score on ground truth',
#             'extraction time'
#             ]
# print(df.to_string(index=False))